# Preparação dos Documentos

In [1]:
import PyPDF2
import chromadb
import uuid
import os
import openai
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from getpass import getpass

In [2]:
# Definição do tamanho dos blocos no ChromaDB
CHUNK_SIZE = 1000
OFFSET = 200

# Conexão com a API da OpenAI
with open('G:/Drives compartilhados/RISCO E COMPLIANCE/Relatórios de Risco/Risco/FIDCS/SCRIPTS_RISCO/Projeto IA/OPENAI_API_KEY.txt', 'r', encoding='utf-8') as file:
    openai_key = file.read()

openai_client = OpenAI(api_key= os.environ["OPENAI_API_KEY"])
# openai_client = OpenAI(api_key= openai_key)
# openai.api_key = os.getenv('OPENAI_API_KEY')

# Conexão com o cliente do banco de dados ChromaDB
chromadb_path = "G:/Drives compartilhados/RISCO E COMPLIANCE/Relatórios de Risco/Risco/FIDCS/SCRIPTS_RISCO/Projeto IA/Base ChromaDB/"
chroma_client = chromadb.PersistentClient(path= chromadb_path)
collection    = chroma_client.get_or_create_collection(name= "Regulamentos_FIDC_PLN")

In [3]:
# Checa todo o conteúdo inserido no ChromaDB
collection.get()

{'ids': ['1b7358a8-6bc9-4b35-a7dd-2b121b7ba07f',
  '8a2a82ab-ac60-4492-b652-f9714384e507',
  'aa052f41-cebe-466c-afd1-346a09b89890',
  'f697d68d-2c34-4b8f-86db-56d990fc2469',
  '50b984c2-bed3-4ba4-9104-a2030a64c2ef',
  'e5efd7e9-4070-447d-9f98-c3b94ea80279',
  '90bddcfd-ab88-44db-9a89-259fef839af4',
  '646a4c08-bfb2-40d7-81fc-b98008a31d2a',
  '2a6b0aa1-c98d-4d7c-882d-4bbc0078652b',
  '80ea8fb4-e3c5-4aab-8ff7-d3c5119adb3d',
  '628e8790-9297-4992-844f-4cfe9130e267',
  'efa9a8b7-8228-4e7e-a292-e691be089d20',
  '59336941-9a64-470c-bcdf-a825d9ce3e21',
  'fd44754a-306e-423b-9473-c0fde5554953',
  '238495d1-e8c7-4758-b20d-471bd1cfb8c3',
  'a3435a08-d476-4ce2-b282-d4966edfbc9b',
  '93a3de5f-3e27-4a8d-a9ed-65ed9da37d75',
  'e4b8fb1f-cf67-4a9a-bd09-03a9fda1c1d4',
  'a995eaf0-4f7b-4853-b8d2-ed0740165d4d',
  'd14d9e37-f559-4f9c-9874-37ccba5b330d',
  'fcc1b1c5-6348-4bfb-ab58-9b681c3054f3',
  '8a8e4172-6630-480b-a17c-c267c5f42c6e',
  'a26c0ec3-41d6-4242-ad98-f055bc581930',
  'd0ac6c6d-4610-49e5-99a1-

In [7]:
# Recebe um documento PDF e retorna seu conteúdo em uma String
def get_document(document_path):
    file = open(document_path, 'rb')
    reader = PyPDF2.PdfReader(file)

    document_text = ""
    for i in range(len(reader.pages)):
        page = reader.pages[i]
        content = page.extract_text()
        document_text += content

    len(document_text)

    return document_text

# Divide o conteúdo em String do documento em partes do tamanho CHUNK_SIZE
def split_document(document_text):
    documents = []
    for i in range(0, len(document_text), CHUNK_SIZE):
        start = i
        end = i + CHUNK_SIZE
        if start != 0:
            start = start - OFFSET
            end =  end - OFFSET
        documents.append(document_text[start: end])
    return documents

# Chamada da API OpenAI para transformação do texto em vetor utilizando text-embedding
def get_embedding(text):

    embedding = OpenAIEmbeddings(
        model= "text-embedding-ada-002",
        chunk_size= CHUNK_SIZE
    )

    emb = embedding.embed_query(text)

    return emb

# Preparação do embedding e metadados dos documentos
def prepare_documents(documents, document_name):
    embeddings = []
    metadatas = []
    for i, doc in enumerate(documents):
        embeddings.append(get_embedding(doc))
        metadatas.append({"source": document_name, "partition" : i})

    return embeddings, metadatas

# Cria uma chave primária de ID para cada documento
def create_ids(documents):
    return [uuid.uuid4() for _ in documents]

# Inserção dos dados preparados no ChromaDB
def insert_data(documents, embeddings, metadatas, ids):

    collection.add(
        embeddings= embeddings,
        documents=  documents,
        metadatas=  metadatas,
        ids= ids
    )

    print(f"Dados inseridos com sucesso! {len(documents)} Chunks")

In [19]:
documents = []
embeddings = []
metadatas = []

def run():
    print("Preparando Documentos...")
    data_path = 'G:/Drives compartilhados/GESTAO/_Operacional/Planilhas Gestão/Scripts/temp/temp_regulamentos'

    documents_names = [f for f in os.listdir(data_path) if f.endswith('.pdf')]
    documents_names_size = len(documents_names)

    for i, document_name in enumerate(documents_names):

        print(f"{i+1}/{documents_names_size}: {document_name}")

        document = get_document(os.path.join(data_path, document_name))
        document_chunks = split_document(document)
        document_embeddings, document_metadatas = prepare_documents(document_chunks, document_name)
        documents.extend(document_chunks)
        embeddings.extend(document_embeddings)
        metadatas.extend(document_metadatas)

    ids = create_ids(documents)
    ids = [str(uuid.uuid4()) for _ in documents]

    insert_data(documents, embeddings, metadatas, ids)

if __name__ == "__main__":
    #run()
    pass

In [ ]:
# Checa todo o conteúdo inserido no ChromaDB
collection.get()

# Implementação da RAG

In [6]:
# Função que pesquisa pelos documentos relevantes baseando-se no contexto da pergunta
def search_document(question):

    print("Pesquisando documentos relevantes...")

    query_embedding = get_embedding(question)
    results = collection.query(
        query_embeddings= [query_embedding],
        n_results= 3
    )

    return results

# Função auxiliar que formata a lista de documentos para uma String
def format_search_result(relevant_documents):

    formatted_list = []
    for i, doc in enumerate(relevant_documents["documents"][0]):
        formatted_list.append("[{}]: {}".format(relevant_documents["metadatas"][0][i]["source"], doc))

    documents_str = "\n".join(formatted_list)

    print(documents_str)

    return documents_str

# Aplicação do LLM

In [8]:
question = """Acesse todos os documentos em sua base e pesquise quais deles incluem os seguintes termos:
'Saque aniversário', 'FGTS', 'INSS', 'Consignado', 'Saque-Aniversário FGTS'.
Retorne uma lista com todos os documentos únicos que possuírem qualquer um destes termos citados pelo menos uma vez."""

relevant_documents = search_document(question)
documents_str = format_search_result(relevant_documents)

Pesquisando documentos relevantes...
[MORE 051 RESP LIMITADA FICFIDC ÚNICA 1.pdf]: AS DE FUNDOS DE 
INVESTIMENTO EM  DIREITOS CREDITÓRIOS  RESPONSABILIDADE LIMITADA  
CNPJ nº   
BTG Pactual  
SAC: 0800 772 28 27 - Ouvidoria: 0800 722 00 48 - btgpactual.com  3 
 
processo de recuperação judicial ou extrajudicial, desde que cumulativamente atend am aos seguintes 
requisitos: (a) não sejam originados por contratos mercantis de compra e venda de produtos, mercadorias e 
serviços para entrega ou prestação futura; e (b) a sociedade esteja sujeita a plano de recuperação homologado 
em juízo, independentem ente do trânsito em julgado da homologação do plano de recuperação judicial ou 
extrajudicial; e (ii) os precatórios federais, desde que cumulativamente atendam aos seguintes requisitos: a) 
não apresentem nenhuma impugnação, judicial ou não; e (b) já tenham  sido expedidos e remetidos ao Tribunal 
Regional Federal competente;  
“Documentos Comprobatórios ”: documentação  necessária  para o 

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model = "gpt-4o-mini", temperature= 0, max_tokens= 4096)

prompt = """Você é um assistente de IA que responde as dúvidas dos usuários com bases nos documentos abaixo.
Os documentos abaixo apresentam as fontes atualizadas e devem ser consideradas como verdade. Sempre que uma
pergunta oferecer o nome de um FIDC, você deve procurar pela melhor correspondência deste nome na lista de documentos.
Cite a fonte quando fornecer a informação.
Documentos:
{documents}
""" #Caso o nome citado na pergunta não exista na lista de documentos, responda que ele não foi encontrado.

prompt = prompt.format(documents= documents_str)

messages=[
  SystemMessage(content= prompt),
  HumanMessage(content= question)
]

In [10]:
answer = modelo.invoke(messages)

In [11]:
answer.content

"Com base nos documentos disponíveis, não encontrei menções aos termos 'Saque aniversário', 'FGTS', 'INSS', 'Consignado' ou 'Saque-Aniversário FGTS'. Portanto, não há documentos que incluam qualquer um desses termos."